In [1]:
import requests
import json
import pandas as pd
from config import covid_api
from config import google_api
import gmaps

# NOTES:
# https://api.covid19api.com/world/total
# This is all data - returns 10MB+
# https://api.covid19api.com/all


In [2]:
# From COVID-19 API
# https://covid19api.com/

#base_url = 'https://api.covid19api.com/summary'
#headers = {"X-Access-Token":covid_api}

#response = requests.get(base_url, headers=headers).json()
#print(json.dumps(response, indent=4, sort_keys=True))
#response['Countries']

In [3]:
# Import CRE csv from
# https://www2.census.gov/data/experimental-data-products/community-resilience-estimates/2020/

cre_path = "data/cre-2018-a11.csv"
cre_df = pd.read_csv(cre_path)

# Break down by North Carolina
nc_cre_df = cre_df.loc[cre_df['STABREV']=='NC']

# Clean up the columns
nc_cre_cleaned_df = nc_cre_df[['rfgrp','prednum','popuni']]

# Group by risk factors and add up
nc_overall_rf_df = nc_cre_cleaned_df.groupby('rfgrp').sum()

# Build new series to hold percentages
nc_overall_rf_df['percentage'] = ''

# Iterate over rows to calculate percentages and store in the new series 
for index, row in nc_overall_rf_df.iterrows():
    pct = (row['prednum'] / row['popuni'])*100
    nc_overall_rf_df.at[index,'percentage'] = pct

# Display the dataframe
nc_overall_rf_df


,prednum,popuni,percentage
rfgrp,,,
0RF,5793084,20438202,28.3444
1-2RF,9733922,20438202,47.6261
3PLRF,4911196,20438202,24.0295


In [4]:
# Break down NC data by county and risk factor levels
nc_county_rf_df = nc_cre_df.groupby(['ctname','rfgrp']).sum()

# Keep only the series we need
nc_county_rf_df = nc_county_rf_df[['prednum','popuni']]

# Make a new series to hold percentages
nc_county_rf_df['percentage'] = ''

# Iterate over rows to calculate percentages and store in the new series 
for index, row in nc_county_rf_df.iterrows():
    pct = (row['prednum'] / row['popuni'])*100
    nc_county_rf_df.at[index,'percentage'] = pct

# Save as CSV
#nc_county_rf_df.to_csv(r'NC_counties_risk factors.csv')

# Display the dataframe
nc_county_rf_df


prednum  popuni percentage
ctname               rfgrp                            
Alamance County, NC  0RF      96568  323580    29.8436
                     1-2RF   149414  323580    46.1753
                     3PLRF    77598  323580    23.9811
Alexander County, NC 0RF      19578   72192    27.1193
                     1-2RF    34884   72192    48.3211
...                             ...     ...        ...
Yadkin County, NC    1-2RF    33378   74996    44.5064
                     3PLRF    21412   74996    28.5509
Yancey County, NC    0RF       8614   35740    24.1018
                     1-2RF    16542   35740    46.2843
                     3PLRF    10584   35740    29.6139

[300 rows x 3 columns]

In [5]:
counties_path = "data/us-counties.csv"
county_df = pd.read_csv(counties_path)

#county_df

nc_county_df = county_df.loc[county_df['state']=='North Carolina']

nc_county_df


,date,county,state,fips,cases,deaths
483,2020-03-03,Wake,North Carolina,37183.0,1,0
520,2020-03-04,Wake,North Carolina,37183.0,1,0
567,2020-03-05,Wake,North Carolina,37183.0,1,0
630,2020-03-06,Chatham,North Carolina,37037.0,1,0
631,2020-03-06,Wake,North Carolina,37183.0,1,0
...,...,...,...,...,...,...
329952,2020-07-13,Wayne,North Carolina,37191.0,2006,30
329953,2020-07-13,Wilkes,North Carolina,37193.0,628,7
329954,2020-07-13,Wilson,North Carolina,37195.0,942,24
329955,2020-07-13,Yadkin,North Carolina,37197.0,385,5


In [6]:
test_path = "data/CountyCasesDeathsAndPer10KResidents_Extract_MigratedData.csv"
test_df = pd.read_csv(test_path)
test_df


,County,Cases,Cases Per 10000 Residents,Deaths
0,New Hanover,1560,67,8
1,Surry,592,82,3
2,Robeson,1708,130,43
3,Person,134,34,1
4,Alamance,1492,90,37
...,...,...,...,...
95,Wilkes,632,92,7
96,Dare,121,33,1
97,Nash,666,71,6
98,Carteret,144,21,3


In [7]:
test2_path = "data/DailyMetrics_Extract_MigratedData.csv"
test2_df = pd.read_csv(test2_path)
test2_df

,Date,Hospitalizations,NC Daily Test,New Cases by Report Date,New Cases by Specimen Date,Positive Test Percentage
0,4/13/20,313.0,1298.0,296,282.0,11%
1,7/8/20,994.0,25299.0,1435,1887.0,8%
2,4/5/20,261.0,1343.0,183,86.0,8%
3,3/18/20,NaN,205.0,22,150.0,NaN
4,6/13/20,823.0,17352.0,1427,685.0,9%
...,...,...,...,...,...,...
130,3/9/20,NaN,NaN,0,NaN,NaN
131,3/6/20,NaN,NaN,1,NaN,NaN
132,3/3/20,NaN,NaN,0,NaN,NaN
133,3/5/20,NaN,NaN,0,NaN,NaN


In [8]:
test3_path = "data/Demographics_Extract_MigratedData.csv"
test3_df = pd.read_csv(test3_path)
test3_df

,Age Group,calc.ValueComma,Demographic,Gender,Hispanic,Name,Race,Value
0,Missing,849,Gender,Missing,NaN,Cases,NaN,849
1,Missing,"45,443",Gender,Female,NaN,Cases,NaN,45443
2,Missing,"43,192",Gender,Male,NaN,Cases,NaN,43192
3,Missing,11,Gender,Missing,NaN,Deaths,NaN,11
4,Missing,719,Gender,Female,NaN,Deaths,NaN,719
...,...,...,...,...,...,...,...,...
75,Missing,1%,Race,Missing,NaN,Percent of Deaths,Asian,1%
76,Missing,0%,Race,Missing,NaN,Percent of Deaths,Native Hawaiian or Pacific Islander,0%
77,Missing,6%,Race,Missing,NaN,Percent of Deaths,Other,6%
78,Missing,NaN,Race,Missing,NaN,Percent of Deaths,Missing,NaN


In [9]:
test4_path = "data/ZIPCodeCasesDeathsAndPer10KResidents_Extract_MigratedData.csv"
test4_df = pd.read_csv(test4_path)
test4_df

,ZIP Code,Cases,Cases Per 10000 Residents,Deaths,Number of Records
0,27311,5,16.0,0,1
1,27919,0,0.0,0,1
2,28698,1,5.0,0,1
3,27962,23,31.0,0,1
4,28345,100,79.0,3,1
...,...,...,...,...,...
775,27704,949,241.0,13,1
776,27545,370,122.0,18,1
777,27231,5,24.0,0,1
778,28152,151,58.0,0,1


In [10]:
# COVID testing location data
# From https://github.com/codersagainstcovidorg/covid19testing-backend

testing_df = pd.DataFrame(columns=['Name','Lat','Lon','City','State','Collecting','Evaluating'])
testing_df

base_url = "http://api.findcovidtesting.com/api/v1/location"
response = requests.get(base_url).json()

In [11]:
# Use this to see how the response is structured
# Relevant info seems to be:
# location_name
# is_collecting_samples - boolean
# is_evaluating_symptoms - boolean
# location_address_locality = city
# location_address_region = state
# location_latitude
# location_longitude

#response[0]

In [12]:
# Iterate through the response and store selected variables in dataframe

for i in range(len(response)):
    if response[i]['location_address_region'] == 'NC':
        testing_df.loc[i,'Name'] = response[i]['location_name']
        testing_df.loc[i,'Lat'] = response[i]['location_latitude']
        testing_df.loc[i,'Lon'] = response[i]['location_longitude']
        testing_df.loc[i,'City'] = response[i]['location_address_locality']
        testing_df.loc[i,'State'] = response[i]['location_address_region']
        testing_df.loc[i,'Collecting'] = response[i]['is_collecting_samples']
        testing_df.loc[i,'Evaluating'] = response[i]['is_evaluating_symptoms']

# Display the dataframe       
testing_df

,Name,Lat,Lon,City,State,Collecting,Evaluating
6,AppHealthCare - Alleghany Health Center,36.4924,-81.1459,Sparta,NC,True,True
18,Drive IV Hydration and MedSpa,34.791,-77.4177,Jacksonville,NC,True,True
42,Avance Care,35.6313,-78.5021,Clayton,NC,True,True
68,Stanly County Health Department,35.3658,-80.1977,Albermarie,NC,True,True
79,Carver Library and FTCC Mazie Woodruff Center ...,36.1504,-80.2124,Winston Salem,NC,True,True
...,...,...,...,...,...,...,...
9737,Dosher Urgent Care,33.9335,-78.0665,Brunswick County,NC,True,True
9783,Windsor Center (Guilford County Division of Pu...,36.0643,-79.7705,,NC,True,False
9853,Medac Urgent Care (Monkey Junction),34.1429,-77.8922,New Hanover,NC,True,True
9864,Macon County Public Health,35.1969,-83.3715,Franklin,NC,True,True


In [13]:
gmaps.configure(api_key=google_api)

# Build the map
# Set center and zoom level
fig = gmaps.figure(map_type='HYBRID',center=(35.3,-79.5), zoom_level=7)
locations = testing_df[['Lat', 'Lon']]
hover = testing_df['Name'].tolist()

symbols = gmaps.symbol_layer(locations, fill_color='white', stroke_color='white', scale = 2, info_box_content = hover)
fig.add_layer(symbols)


# Display the map
fig

Figure(layout=FigureLayout(height='420px'))